# Self-Driving Car Engineer Nanodegree
[![Udacity - Self-Driving Car NanoDegree](https://s3.amazonaws.com/udacity-sdc/github/shield-carnd.svg)](http://www.udacity.com/drive)

## Project: **Advanced Lane Finding** 

## 3. Pipeline (video)

### Import Statements

In [1]:
import os
import pickle as pl
import cv2
import numpy as np

#import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

#import pipeline functions
from pipeline_functions import *

### Initialisation

In [2]:
#create output directories if not existing
for path in ["output_videos"]:

    if not os.path.exists(path):
        os.makedirs(path)
        
#import all necessary params from params directory
for file in os.listdir("params/"):
        if file[-3:]=='.pl':
            paramname = file[:-3]

            with open ("params/"+file, 'rb') as f:
                globals()[paramname] = pl.load(f)

### Apply pipeline to all example videos

In [3]:
project_output_extract = 'output_videos/project_video_extract.mp4'
project_output_rewarped = 'output_videos/project_video_rewarped.mp4'

#clip1 = VideoFileClip("input_videos/project_video.mp4").subclip(0, 5)
clip1 = VideoFileClip("input_videos/project_video.mp4")

project_clip_extract = clip1.fl_image(extract_original_pixels)
%time project_clip_extract.write_videofile(project_output_extract, audio=False)

project_clip_rewarped = clip1.fl_image(show_rewarped_fitted_poly)
%time project_clip_rewarped.write_videofile(project_output_rewarped, audio=False)

[MoviePy] >>>> Building video output_videos/project_video_extract.mp4
[MoviePy] Writing video output_videos/project_video_extract.mp4


100%|█████████▉| 1260/1261 [03:23<00:00,  6.19it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_videos/project_video_extract.mp4 

CPU times: user 2min 2s, sys: 42.8 s, total: 2min 45s
Wall time: 3min 23s
[MoviePy] >>>> Building video output_videos/project_video_rewarped.mp4
[MoviePy] Writing video output_videos/project_video_rewarped.mp4


100%|█████████▉| 1260/1261 [03:43<00:00,  5.59it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_videos/project_video_rewarped.mp4 

CPU times: user 2min 25s, sys: 36.4 s, total: 3min 1s
Wall time: 3min 43s


In [4]:
challenge_output_extract = 'output_videos/challenge_video_extract.mp4'
challenge_output_rewarped = 'output_videos/challenge_video_rewarped.mp4'

#clip2 = VideoFileClip("input_videos/challenge_video.mp4").subclip(4.68, 4.7)
#clip2 = VideoFileClip("input_videos/challenge_video.mp4").subclip(4.78, 4.8)
#clip2 = VideoFileClip("input_videos/challenge_video.mp4").subclip(3, 6)
clip2 = VideoFileClip("input_videos/challenge_video.mp4")


challenge_clip_extract = clip2.fl_image(extract_original_pixels)
%time challenge_clip_extract.write_videofile(challenge_output_extract, audio=False)

challenge_clip_rewarped = clip2.fl_image(show_rewarped_fitted_poly)
%time challenge_clip_rewarped.write_videofile(challenge_output_rewarped, audio=False)

[MoviePy] >>>> Building video output_videos/challenge_video_extract.mp4
[MoviePy] Writing video output_videos/challenge_video_extract.mp4


100%|██████████| 485/485 [01:17<00:00,  6.53it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_videos/challenge_video_extract.mp4 

CPU times: user 47 s, sys: 16.8 s, total: 1min 3s
Wall time: 1min 18s
[MoviePy] >>>> Building video output_videos/challenge_video_rewarped.mp4
[MoviePy] Writing video output_videos/challenge_video_rewarped.mp4


100%|██████████| 485/485 [01:20<00:00,  6.29it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_videos/challenge_video_rewarped.mp4 

CPU times: user 50.5 s, sys: 14.3 s, total: 1min 4s
Wall time: 1min 20s


In [5]:
harder_challenge_output_extract = 'output_videos/harder_challenge_video_extract.mp4'
harder_challenge_output_rewarped = 'output_videos/harder_challenge_video_rewarped.mp4'

#clip3 = VideoFileClip("input_videos/harder_challenge_video.mp4").subclip(10, 15)
clip3 = VideoFileClip("input_videos/harder_challenge_video.mp4")

harder_challenge_clip_extract = clip3.fl_image(extract_original_pixels)
%time harder_challenge_clip_extract.write_videofile(harder_challenge_output_extract, audio=False)

harder_challenge_clip_rewarped = clip3.fl_image(show_rewarped_fitted_poly)
%time harder_challenge_clip_rewarped.write_videofile(harder_challenge_output_rewarped, audio=False)

[MoviePy] >>>> Building video output_videos/harder_challenge_video_extract.mp4
[MoviePy] Writing video output_videos/harder_challenge_video_extract.mp4


100%|█████████▉| 1199/1200 [03:25<00:00,  5.76it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_videos/harder_challenge_video_extract.mp4 

CPU times: user 1min 58s, sys: 40.2 s, total: 2min 39s
Wall time: 3min 26s
[MoviePy] >>>> Building video output_videos/harder_challenge_video_rewarped.mp4
[MoviePy] Writing video output_videos/harder_challenge_video_rewarped.mp4


100%|█████████▉| 1199/1200 [03:45<00:00,  5.57it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_videos/harder_challenge_video_rewarped.mp4 

CPU times: user 2min 27s, sys: 34.8 s, total: 3min 1s
Wall time: 3min 45s


### implement smooting into the pipeline

In [6]:
class ProcessImageStream:
    def __init__(self):
        self.Reliability_l = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        self.Reliability_r = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        
        self.Leftfit = [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]
        self.Rightfit = [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]
        
        self.crv = 1000
        self.lastReliableWidth_px = 750
        
        
    def __call__(self, img):
        #correct image distortion
        img = cv2.undistort(img, mtx, dist, None, None)
        
        #if the lane is very curvy, a larger region of interest can halp to detect the markings correctly
        if self.crv<150:
            #extract binary mask
            roi = np.array([[(500, 410), (810, 410), (1200, 700), (170, 700)]])
            mask = extract_binary_mask(img, roi)
            
        else:
            #extract binary mask
            mask = extract_binary_mask(img)

        #unwarp the binary mask
        unwarped = unwarp(mask, M)

        #calculate image with fitted polynomials
        rewarped_out_img, reliability_l, reliability_r, left_fit, right_fit, width, crv = \
                                            visualize_polyfit(unwarped, colorize_markings=True,\
                                                              show_windows=False, suppress_noise=True,\
                                                              colorize_lane=True, rewarp=True,\
                                                              show_crv=True, verbose=True,\
                                                              history=[self.Reliability_l, self.Reliability_r,\
                                                                      self.Leftfit, self.Rightfit,\
                                                                      self.lastReliableWidth_px])
        
        #append newly obtained relibilty and polynomial values to history
        self.Reliability_l.pop(0)
        self.Reliability_l.append(reliability_l)
        
        self.Reliability_r.pop(0)
        self.Reliability_r.append(reliability_r) 
        
        self.Leftfit.pop(0)
        self.Leftfit.append(left_fit)
        
        self.Rightfit.pop(0)
        self.Rightfit.append(right_fit)
        
        #adapt last reliable width value and curvature
        xpix_per_m = 820.0/3.7 
        if reliability_l>0.15 and reliability_r>0.15 and width>2.2 and width<5.2:
            self.lastReliableWidth_px = width*xpix_per_m
            
        self.crv = crv
        
        #overlay original image with detections and return it
        final = np.copy(img)
        cv2.addWeighted(final, 0.5, rewarped_out_img, 1.0, 0.0, final)
        
        return final

In [7]:
project_output_final = 'output_videos/project_video_final.mp4'
#project_clip = VideoFileClip("input_videos/project_video.mp4").subclip(40, 42)
#project_clip = VideoFileClip("input_videos/project_video.mp4").subclip(37, 43)
#project_clip = VideoFileClip("input_videos/project_video.mp4").subclip(27, 32)
project_clip = VideoFileClip("input_videos/project_video.mp4")
project_clip_final = project_clip.fl_image(ProcessImageStream())
%time project_clip_final.write_videofile(project_output_final, audio=False)

[MoviePy] >>>> Building video output_videos/project_video_final.mp4
[MoviePy] Writing video output_videos/project_video_final.mp4


100%|█████████▉| 1260/1261 [04:53<00:00,  4.29it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_videos/project_video_final.mp4 

CPU times: user 2min 30s, sys: 38.4 s, total: 3min 8s
Wall time: 4min 56s


In [8]:
challenge_output_final = 'output_videos/challenge_video_final.mp4'
#challenge_clip = VideoFileClip("input_videos/challenge_video.mp4").subclip(4, 6)
#challenge_clip = VideoFileClip("input_videos/challenge_video.mp4").subclip(0, 1)
#challenge_clip = VideoFileClip("input_videos/challenge_video.mp4").subclip(0, 0.02)
#challenge_clip = VideoFileClip("input_videos/challenge_video.mp4").subclip(5, 6)
challenge_clip = VideoFileClip("input_videos/challenge_video.mp4")
challenge_clip_final = challenge_clip.fl_image(ProcessImageStream())
%time challenge_clip_final.write_videofile(challenge_output_final, audio=False)

[MoviePy] >>>> Building video output_videos/challenge_video_final.mp4
[MoviePy] Writing video output_videos/challenge_video_final.mp4


100%|██████████| 485/485 [01:45<00:00,  4.63it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_videos/challenge_video_final.mp4 

CPU times: user 52 s, sys: 15.4 s, total: 1min 7s
Wall time: 1min 47s


In [9]:
harder_challenge_output_final = 'output_videos/harder_challenge_video_final.mp4'
harder_challenge_clip = VideoFileClip("input_videos/harder_challenge_video.mp4")
#harder_challenge_clip = VideoFileClip("input_videos/harder_challenge_video.mp4").subclip(0, 10)
harder_challenge_clip_final = harder_challenge_clip.fl_image(ProcessImageStream())
%time harder_challenge_clip_final.write_videofile(harder_challenge_output_final, audio=False)

[MoviePy] >>>> Building video output_videos/harder_challenge_video_final.mp4
[MoviePy] Writing video output_videos/harder_challenge_video_final.mp4


100%|█████████▉| 1199/1200 [05:28<00:00,  3.92it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_videos/harder_challenge_video_final.mp4 

CPU times: user 2min 30s, sys: 37.1 s, total: 3min 8s
Wall time: 5min 31s
